# Deep learning example with Keras and Tensorflow

## 1. Data generation
We can load a training and validation set using the ImageDataGenerator class from Keras. This ImageDataGenerator object allows to build a data flow that loads a directory of images lazily. In addition, data can be augmented on the fly to increase nuance in the datasets.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'data/train'
test_dir = 'data/test'
inference_dir = 'data/inference'
im_height = 128
im_width = 128
im_channels = 3
batch_size = 5

train_datagenerator = ImageDataGenerator(rescale=1./255,
                                         horizontal_flip=True,
                                         rotation_range=10,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_dataflow = train_datagenerator.flow_from_directory(train_dir,
                                                         target_size=(image_height, image_width),
                                                         batch_size=batch_size)
test_dataflow = test_datagenerator.flow_from_directory(test_dir,
                                                       target_size=(image_height, image_width),
                                                       batch_size=batch_size)

print("Classes of the model with their one-hot labels: ", train_dataflow.class_indices)

## 2. Model building

### Multi-layer perceptron
We can try to train a multi-layer perceptron that classifies the images. However, the results will not get much better than random guessing. This is partly due to the inability of this neural network architecture to find valuable features in the high dimensional space of the problem, which is defined by the number of input neurons: `im_height * im_width * im_channels`.

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(im_height, im_width, im_channels)))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
training_history = model.fit_generator(train_dataflow,
                    steps_per_epoch=train_dataflow.samples // batch_size,
                    epochs=10,
                    validation_data=test_dataflow,
                    validation_steps=test_dataflow.samples // batch_size)

plot(training_history)

### Transfer learning with a very deep neural net
[Transfer learning](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.146.1515&rep=rep1&type=pdf) is a way of using pre-trained models and finetuning them for making inferences on new classes. Because large models such as VGG16 or Inception can take days or even weeks to train from scratch, transfer learning is a convenient method that allows us to use some of the best performing models without investing heavily into powerful infrastructure.

[MobileNet](https://research.googleblog.com/2017/06/mobilenets-open-source-models-for.html) is currently one of the best scalable deep learning models. It is built by Google with a specific focus on mobile devices and environments with restrictive resources. This makes the model interesting both for training, as training can be performed even on basic systems, and for inference.

In [ ]:
from keras.engine import Model
from keras.applications import MobileNet

pretrained_model = MobileNet(weights='imagenet', input_shape = (image_height,image_width,3), include_top=False)

for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dropout, Dense

# Add a top layer model that can be trained on new classes. Any classifier will do.
top_model = Sequential()
top_model.add(Flatten(input_shape=pretrained_model.output_shape[1:]))
top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.1))
top_model.add(Dense(2, activation='sigmoid'))

transfer_learning_model = Model(inputs=pretrained_model.input, outputs=top_model(pretrained_model.output))

transfer_learning_model.compile(loss='binary_crossentropy',
                                optimizer='adam',
                                metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('weights.{epoch:02d}-{val_acc:.2f}.h5',
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            save_weights_only=True,
                            period=5)

training_history = transfer_learning_model.fit_generator(train_dataflow,
                    steps_per_epoch=train_dataflow.samples // batch_size,
                    epochs=50,
                    validation_data=test_dataflow,
                    validation_steps=test_dataflow.samples // batch_size,
                    callbacks=[checkpoint])

plot(training_history)

## 3. Using the model for inference
We can now predict the classes of new images using the trained model. The model can however easily be fooled due to the limited coverage of the training and test set over real-world examples.

In [ ]:
hotdog = read_for_inference('hotdog.jpg')
predict(hotdog, transfer_learning_model)

pizza = read_for_inference('pizza_2.jpg')
predict(pizza, transfer_learning_model)

cute_dog = read_for_inference('cute_dog.jpg')
predict(cute_dog, transfer_learning_model)

shoe = read_for_inference('shoe.jpg')
predict(shoe, transfer_learning_model)

## Some useful functions

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize
from keras.preprocessing.image import load_img, img_to_array

def read_for_inference(path):
    return imresize(load_img(os.path.join(inference_dir, path)), (im_height, im_width))/255.

def predict(image, model):
    prediction = model.predict(np.asarray([image]))
    show(image)
    show(load_img('data/class_ims/' + str(np.argmax(prediction)) + '.png'))
    print(prediction)

def show(image):
    plt.imshow(image)
    plt.show()

def plot(training_history):
    plt.plot(training_history.history['acc'])
    plt.plot(training_history.history['val_acc'])
    plt.xlabel('epoch')
    plt.ylabel('model accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()